---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [7]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

# Import libraries for ML:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    # We will use 2 metrics to evaluate the graphs - average clustering coefficient and
    # average shortest path, alongside a linear correlation value.
    # Create list to store result:
    result = []
    # Loop over all graphs:
    for graph in P1_Graphs:
        ## Preferential Attachment ##
        
        # Evaluating linear correlation between degree and number of nodes in log scale:
        degrees = graph.degree()
        degree_values = sorted(set(degrees.values()))
        node_quantities = [list(degrees.values()).count(i)/float(nx.number_of_nodes(graph)) for i in degree_values]
        # Pearson correlation matrix:
        R = np.corrcoef(np.log(degree_values), np.log(node_quantities))
        # If the linear correlation is high, it is a PA graph:
        if abs(R[0][1]) >= 0.89:
            result.append('PA')
        else:
            ## Small World ##
            
            # Compute average clustering coefficient:
            avg_cc = nx.average_clustering(graph)
            # Compute average shortest path:
            avg_sp = nx.average_shortest_path_length(graph)
            # Evaluate type of SW - low p gives high average clustering coefficient:
            if avg_sp < 10 and avg_cc > 0.2:
                result.append('SW_L')
            else:
                result.append('SW_H')

    # Return list with result of analysis:
    return result

# Testing:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [12]:
def salary_predictions():
    ## Feature Extraction - Node Attributes & Centrality Measures ##
    
    # Create dataframe to store graph features with nodes as index:
    features_df = pd.DataFrame(index=G.nodes())
    
    # Get info of department and management salary and store:
    features_df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    features_df['management_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    # Compute and store degree of nodes:
    features_df['degree'] = pd.Series(G.degree())
    # Compute and store clustering coefficient of nodes:
    features_df['clust_coeff'] = pd.Series(nx.clustering(G))
    # Compute and store degree centrality of nodes:
    features_df['degree_cent'] = pd.Series(nx.degree_centrality(G))
    # Compute and store closeness centrality of nodes:
    features_df['close_cent'] = pd.Series(nx.closeness_centrality(G))
    # Compute and store normalized betweenness centrality of nodes excluding endpoints:
    features_df['btwn_cent'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    
    
    ## Preprocessing ##
    
    # Split in train and test data based on management salary:
    train_df = features_df[features_df['management_salary'].notnull()]
    test_df = features_df[pd.isnull(features_df['management_salary'])]
    
    # Split features from labels:
    X_train_full = train_df.drop(['management_salary'], axis = 1)
    y_train_full = train_df['management_salary']
    
    X_test = test_df.drop(['management_salary'], axis = 1)
    # No need for y_test since all data will be NaN
    
    # Since the test set is a pure prediction, we need to split the
    # training set to be able to evaluate our model.
    # Split data for training and validation:
    X_train, X_validate, y_train, y_validate = train_test_split(X_train_full, y_train_full, random_state=0)
    
    
    ## Gradient Boost Classification Model ##
    
    # Create model and fit on training data:
    gboost = GradientBoostingClassifier().fit(X_train, y_train)
    
    
    ## Validation Evaluation - AUC ##
    
    # Calculate decision function for validation data:
    decision_val = gboost.decision_function(X_validate)
    # Obtain ROC curve for validation data:
    fpr_val, tpr_val, _ = roc_curve(y_validate, decision_val)
    # Calculate AUC for validation data:
    auc_val = auc(fpr_val, tpr_val)
    # Result: AUC = 0.9312368972746331
    
    ## Prediction on Test ##
    
    # Calculate decision function for test data:
    decision_test = gboost.decision_function(X_test)
    
    # Return test set prediction probabilities as a series:
    return pd.Series(decision_test, index=X_test.index.values)

# Testing:
salary_predictions()

1      -5.190650
2       4.058988
5       4.545070
8      -1.368491
14     -3.485217
18     -4.131526
27     -3.610389
30      2.625269
31     -1.381721
34     -4.572746
37     -3.951486
40     -3.944509
45     -4.806367
54     -0.569824
55      1.088505
60     -1.954883
62      4.545070
65      4.150036
77     -4.273237
79     -4.839072
97     -5.105997
101    -6.130085
103     0.081893
108    -3.874580
113    -3.819155
122    -6.592781
141    -1.528496
142     4.545070
144    -3.256695
145     0.355360
          ...   
913    -5.049351
914    -4.971673
915    -6.637671
918    -4.148465
923    -4.704026
926    -4.242933
931    -5.556157
934    -7.026856
939    -6.729670
944    -6.860813
945    -4.704026
947    -3.881112
950    -3.420695
951    -5.404176
953    -6.286395
959    -6.729670
962    -6.729670
963    -3.346962
968    -3.817780
969    -3.651737
974    -3.560364
984    -6.860813
987    -2.965761
989    -3.881112
991    -4.344651
992    -6.729670
994    -6.978760
996    -6.7063

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [13]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [16]:
def new_connections_predictions():
    ## Feature Extraction - Link Prediction ##
    
    # Compute and store jaccard coefficients of pair of nodes:
    future_connections['jaccard'] =  [item[2] for item in nx.jaccard_coefficient(G, ebunch=future_connections.index)]
    # Compute and store resource allocation index of pair of nodes:
    future_connections['ra'] =  [item[2] for item in nx.resource_allocation_index(G, ebunch=future_connections.index)]
    # Compute and store preferential attachment score of pair of nodes:
    future_connections['pa'] =  [item[2] for item in nx.preferential_attachment(G, ebunch=future_connections.index)]
    # Compute and store community common neighbours of pair of nodes:
    future_connections['cn_hop'] =  [item[2] for item in nx.cn_soundarajan_hopcroft(G, ebunch=future_connections.index, community='Department')]
    
    
    ## Preprocessing ##
    
    # Split in train and test data based on future connection:
    train_df = future_connections[future_connections['Future Connection'].notnull()]
    test_df = future_connections[pd.isnull(future_connections['Future Connection'])]
    
    # Split features from labels:
    X_train_full = train_df.drop(['Future Connection'], axis = 1)
    y_train_full = train_df['Future Connection']
    
    X_test = test_df.drop(['Future Connection'], axis = 1)
    # No need for y_test since all data will be NaN
    
    # Since the test set is a pure prediction, we need to split the
    # training set to be able to evaluate our model.
    # Split data for training and validation:
    X_train, X_validate, y_train, y_validate = train_test_split(X_train_full, y_train_full, random_state=0)
    
    
    ## Gradient Boost Classification Model ##
    
    # Create model and fit on training data:
    gboost = GradientBoostingClassifier().fit(X_train, y_train)
    
    
    ## Validation Evaluation - AUC ##
    
    # Calculate decision function for validation data:
    decision_val = gboost.decision_function(X_validate)
    # Obtain ROC curve for validation data:
    fpr_val, tpr_val, _ = roc_curve(y_validate, decision_val)
    # Calculate AUC for validation data:
    auc_val = auc(fpr_val, tpr_val)
    # Result: AUC = 0.91341394983788127
    
    ## Prediction on Test ##
    
    # Calculate decision function for test data:
    decision_test = gboost.decision_function(X_test)
    
    # Return test set prediction probabilities as a series:
    return pd.Series(decision_test, index=X_test.index.values)

# Testing:
new_connections_predictions()

(107, 348)   -3.462868
(542, 751)   -4.316788
(20, 426)     0.180446
(50, 989)    -4.316788
(942, 986)   -4.299317
(324, 857)   -4.316788
(13, 710)    -1.840228
(19, 271)    -1.743766
(319, 878)   -4.316788
(659, 707)   -4.316788
(49, 843)    -4.316788
(208, 893)   -4.316788
(377, 469)   -4.246530
(405, 999)   -4.050125
(129, 740)   -4.339823
(292, 618)   -3.926754
(239, 689)   -4.316788
(359, 373)   -4.246530
(53, 523)    -2.424290
(276, 984)   -4.316788
(202, 997)   -4.316788
(604, 619)   -2.571147
(270, 911)   -4.316788
(261, 481)   -2.580846
(200, 450)    5.289255
(213, 634)   -4.316788
(644, 735)   -2.904792
(346, 553)   -4.316788
(521, 738)   -4.316788
(422, 953)   -4.101826
                ...   
(672, 848)   -4.316788
(28, 127)     3.825755
(202, 661)   -4.316788
(54, 195)     6.117887
(295, 864)   -4.316788
(814, 936)   -4.316788
(839, 874)   -4.299317
(139, 843)   -4.316788
(461, 544)   -4.316788
(68, 487)    -4.316788
(622, 932)   -4.316788
(504, 936)   -4.076280
(479, 528) 